数据预处理

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
 
path = "/content/gdrive/My Drive/Independent"
os.chdir(path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# 获取文本
import json

users_label = [] # 用户标签
users_posts = [] # 用户发言
index = 0    # 用户序号

# 读取文本
def read_text(file_path):
    global index

    text = open(file_path, 'r', encoding='UTF-8')
    line = text.readline()
    while line:
        if (len(line.strip()) != 0):
            setting = json.loads(line[1:-2])
            posts = setting['posts']
            label = setting['label']

            add_lable(label)
            add_posts(posts)

            index = index + 1
            line = text.readline()

        if (index == 1000):
            break

    return users_label, users_posts

# 记录用户标签
def add_lable(label):
    if (label == "depression"):
        users_label.append(1)
    else:
        users_label.append(0)

# 记录用户发言
def add_posts(posts):
    user_posts = ""
    for post in posts:
        user_posts = user_posts + " " + post[1]
    users_posts.append(user_posts)

In [ ]:
# 预处理
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# 词频
def count_vec(users_posts):
    count = CountVectorizer(stop_words='english')
    # 计算各个词语出现的次数
    users_posts = count.fit_transform(users_posts)
    return users_posts

# TF-IDF
def tf_idf(users_posts):
    tfidf = TfidfVectorizer(stop_words='english')
    users_posts = tfidf.fit_transform(users_posts)
    return users_posts


In [ ]:
# 保存
from scipy import sparse

def save(users_posts, users_label):
    sparse.save_npz("./result/v_count_posts.npz", users_posts, False)

    file = open("./result/v_label.txt", "w")
    for user_label in users_label:
        file.write(str(user_label) + "\n")
    file.close()

def read_list(path):
    result = []
    with open(path, "r") as f:
        for line in f:
            result.append(line.strip("\n"))

    return result

def read_npz(path):
    sparse_matrix = sparse.load_npz(path)
    return sparse_matrix

In [ ]:
import numpy as np

# 获取用户标签和发言
file_path = "./docs/validation"
users_label, users_posts = read_text(file_path)

# 文本向量化
users_posts = count_vec(users_posts)
#users_posts = tf_idf(users_posts)

# 保存至本地
save(users_posts, users_label)